In [65]:
import pandas as pd
import numpy as np

Считаем датасет. Дропнем не нужные колонки.

In [66]:
df = pd.read_csv('api.csv')
df = df.drop(columns='Unnamed: 0')
df = df.drop(columns='reaction_null')
df.head()

,channel,text,views,date,comments,media,reaction_😢,reaction_🔥,reaction_🤨,reaction_🤡,...,tag_Роснефть,tag_ФСБ,tag_Армия,tag_Путин,tag_Спорт,tag_Интернет,tag_НАТО,tag_Цены,tag_ООН,tag_Искусственный интеллект
0,@novosti_voinaa,У Илона Маска появился 14-й ребенок. Еще одног...,52121.0,2025-03-01 11:40:08,49.0,PHOTO,0,3,4,0,...,0,0,0,0,0,0,0,0,0,1
1,@novosti_voinaa,"Восстановить бонусные карты IKEA, ZARA, LUSH и...",80479.0,2025-03-01 10:18:01,21.0,PHOTO,10,1,61,0,...,0,0,0,0,0,0,0,0,0,1
2,@novosti_voinaa,И для полноты картины минувшего и надолго запо...,148282.0,2025-03-01 00:09:40,164.0,PHOTO,94,109,36,0,...,0,0,0,0,0,0,0,0,0,1
3,@novosti_voinaa,"Видео, после которого вы начнете крепче завязы...",133541.0,2025-02-28 23:59:01,26.0,VIDEO,82,8,5,0,...,0,0,0,0,0,0,0,0,0,0
4,@novosti_voinaa,NaN,135909.0,2025-02-28 23:29:11,207.0,POLL,0,2,259,0,...,0,0,0,0,0,0,0,0,0,0


Удалим колонки в которых все поля пустые (ошибка обращения к API, непредвиденные ситуации).

In [67]:
tags = df.columns[df.columns.str.startswith("tag_")]
reactions = df.columns[df.columns.str.startswith("reaction_")]
df_cleared = df[(df["text"].str.len() > 0) & (df.views > 0) & (df[tags].fillna(0) != 0).any(axis=1) & (df[reactions].fillna(0) != 0).any(axis=1)]
df_cleared.head()

,channel,text,views,date,comments,media,reaction_😢,reaction_🔥,reaction_🤨,reaction_🤡,...,tag_Роснефть,tag_ФСБ,tag_Армия,tag_Путин,tag_Спорт,tag_Интернет,tag_НАТО,tag_Цены,tag_ООН,tag_Искусственный интеллект
0,@novosti_voinaa,У Илона Маска появился 14-й ребенок. Еще одног...,52121.0,2025-03-01 11:40:08,49.0,PHOTO,0,3,4,0,...,0,0,0,0,0,0,0,0,0,1
1,@novosti_voinaa,"Восстановить бонусные карты IKEA, ZARA, LUSH и...",80479.0,2025-03-01 10:18:01,21.0,PHOTO,10,1,61,0,...,0,0,0,0,0,0,0,0,0,1
2,@novosti_voinaa,И для полноты картины минувшего и надолго запо...,148282.0,2025-03-01 00:09:40,164.0,PHOTO,94,109,36,0,...,0,0,0,0,0,0,0,0,0,1
3,@novosti_voinaa,"Видео, после которого вы начнете крепче завязы...",133541.0,2025-02-28 23:59:01,26.0,VIDEO,82,8,5,0,...,0,0,0,0,0,0,0,0,0,0
5,@novosti_voinaa,Таких адреналиновых официальных переговоров гл...,132824.0,2025-02-28 23:08:49,148.0,PHOTO,0,17,37,0,...,0,0,0,0,0,0,0,0,0,1


Нормализуем значение просмотров, комментариев и реакций.

In [68]:
df_cleared['normalized_views'] = df_cleared.groupby('channel')['views'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
)
df_cleared.head()

C:\Users\Giesoft\AppData\Local\Temp\ipykernel_33688\2883568583.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleared['normalized_views'] = df_cleared.groupby('channel')['views'].transform(


,channel,text,views,date,comments,media,reaction_😢,reaction_🔥,reaction_🤨,reaction_🤡,...,tag_ФСБ,tag_Армия,tag_Путин,tag_Спорт,tag_Интернет,tag_НАТО,tag_Цены,tag_ООН,tag_Искусственный интеллект,normalized_views
0,@novosti_voinaa,У Илона Маска появился 14-й ребенок. Еще одног...,52121.0,2025-03-01 11:40:08,49.0,PHOTO,0,3,4,0,...,0,0,0,0,0,0,0,0,1,0.000000
1,@novosti_voinaa,"Восстановить бонусные карты IKEA, ZARA, LUSH и...",80479.0,2025-03-01 10:18:01,21.0,PHOTO,10,1,61,0,...,0,0,0,0,0,0,0,0,1,0.079088
2,@novosti_voinaa,И для полноты картины минувшего и надолго запо...,148282.0,2025-03-01 00:09:40,164.0,PHOTO,94,109,36,0,...,0,0,0,0,0,0,0,0,1,0.268185
3,@novosti_voinaa,"Видео, после которого вы начнете крепче завязы...",133541.0,2025-02-28 23:59:01,26.0,VIDEO,82,8,5,0,...,0,0,0,0,0,0,0,0,0,0.227074
5,@novosti_voinaa,Таких адреналиновых официальных переговоров гл...,132824.0,2025-02-28 23:08:49,148.0,PHOTO,0,17,37,0,...,0,0,0,0,0,0,0,0,1,0.225074


In [69]:
# Проверка на то, что нормализация просмотров работает
df_cleared[df_cleared.channel == "@Novosty"].normalized_views.max() == df_cleared[df_cleared.channel == "@moscowach"].normalized_views.max()

np.True_

In [70]:
df_cleared['normalized_comments'] = df_cleared.groupby('channel')['comments'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
)
df_cleared.head()

C:\Users\Giesoft\AppData\Local\Temp\ipykernel_33688\4107901601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleared['normalized_comments'] = df_cleared.groupby('channel')['comments'].transform(


,channel,text,views,date,comments,media,reaction_😢,reaction_🔥,reaction_🤨,reaction_🤡,...,tag_Армия,tag_Путин,tag_Спорт,tag_Интернет,tag_НАТО,tag_Цены,tag_ООН,tag_Искусственный интеллект,normalized_views,normalized_comments
0,@novosti_voinaa,У Илона Маска появился 14-й ребенок. Еще одног...,52121.0,2025-03-01 11:40:08,49.0,PHOTO,0,3,4,0,...,0,0,0,0,0,0,0,1,0.000000,0.034757
1,@novosti_voinaa,"Восстановить бонусные карты IKEA, ZARA, LUSH и...",80479.0,2025-03-01 10:18:01,21.0,PHOTO,10,1,61,0,...,0,0,0,0,0,0,0,1,0.079088,0.014482
2,@novosti_voinaa,И для полноты картины минувшего и надолго запо...,148282.0,2025-03-01 00:09:40,164.0,PHOTO,94,109,36,0,...,0,0,0,0,0,0,0,1,0.268185,0.118030
3,@novosti_voinaa,"Видео, после которого вы начнете крепче завязы...",133541.0,2025-02-28 23:59:01,26.0,VIDEO,82,8,5,0,...,0,0,0,0,0,0,0,0,0.227074,0.018103
5,@novosti_voinaa,Таких адреналиновых официальных переговоров гл...,132824.0,2025-02-28 23:08:49,148.0,PHOTO,0,17,37,0,...,0,0,0,0,0,0,0,1,0.225074,0.106445


In [71]:
# Проверка на то, что нормализация комментариев работает
df_cleared[df_cleared.channel == "@Novosty"].normalized_views.max() == df_cleared[df_cleared.channel == "@moscowach"].normalized_views.max()

np.True_

In [72]:
sum_reactions = df_cleared[reactions].sum(axis=1)
df_cleared[reactions] = df_cleared[reactions].div(sum_reactions, axis=0) * 100
df_cleared[reactions] = df_cleared[reactions].fillna(0)
df_cleared[reactions].head()

C:\Users\Giesoft\AppData\Local\Temp\ipykernel_33688\1293883178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleared[reactions] = df_cleared[reactions].div(sum_reactions, axis=0) * 100
C:\Users\Giesoft\AppData\Local\Temp\ipykernel_33688\1293883178.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleared[reactions] = df_cleared[reactions].fillna(0)


,reaction_😢,reaction_🔥,reaction_🤨,reaction_🤡,reaction_🕊,reaction_🎉,reaction_🤣,reaction_😱,reaction_👍,reaction_👎,reaction_❤,reaction_💯,reaction_😁,reaction_🤔,reaction_🤬,reaction_🤯,reaction_🙏
0,0.000000,0.508475,0.677966,0.0,0.0,0.0,0.0,0.0,52.881356,4.237288,3.559322,0.0,31.525424,0.0,1.016949,5.593220,0.0
1,2.178649,0.217865,13.289760,0.0,0.0,0.0,0.0,0.0,47.712418,3.267974,4.575163,0.0,12.418301,0.0,7.625272,8.714597,0.0
2,4.669647,5.414804,1.788376,0.0,0.0,0.0,0.0,0.0,11.823150,1.639344,3.030303,0.0,18.479881,0.0,0.347740,52.806756,0.0
3,7.564576,0.738007,0.461255,0.0,0.0,0.0,0.0,0.0,12.177122,0.645756,3.690037,0.0,0.830258,0.0,0.000000,73.892989,0.0
5,0.000000,1.249082,2.718589,0.0,0.0,0.0,0.0,0.0,25.789860,1.763409,6.980162,0.0,16.458486,0.0,0.514328,44.526084,0.0


In [73]:
df_cleared.head()

,channel,text,views,date,comments,media,reaction_😢,reaction_🔥,reaction_🤨,reaction_🤡,...,tag_Армия,tag_Путин,tag_Спорт,tag_Интернет,tag_НАТО,tag_Цены,tag_ООН,tag_Искусственный интеллект,normalized_views,normalized_comments
0,@novosti_voinaa,У Илона Маска появился 14-й ребенок. Еще одног...,52121.0,2025-03-01 11:40:08,49.0,PHOTO,0.000000,0.508475,0.677966,0.0,...,0,0,0,0,0,0,0,1,0.000000,0.034757
1,@novosti_voinaa,"Восстановить бонусные карты IKEA, ZARA, LUSH и...",80479.0,2025-03-01 10:18:01,21.0,PHOTO,2.178649,0.217865,13.289760,0.0,...,0,0,0,0,0,0,0,1,0.079088,0.014482
2,@novosti_voinaa,И для полноты картины минувшего и надолго запо...,148282.0,2025-03-01 00:09:40,164.0,PHOTO,4.669647,5.414804,1.788376,0.0,...,0,0,0,0,0,0,0,1,0.268185,0.118030
3,@novosti_voinaa,"Видео, после которого вы начнете крепче завязы...",133541.0,2025-02-28 23:59:01,26.0,VIDEO,7.564576,0.738007,0.461255,0.0,...,0,0,0,0,0,0,0,0,0.227074,0.018103
5,@novosti_voinaa,Таких адреналиновых официальных переговоров гл...,132824.0,2025-02-28 23:08:49,148.0,PHOTO,0.000000,1.249082,2.718589,0.0,...,0,0,0,0,0,0,0,1,0.225074,0.106445


In [74]:
df_prepared = df_cleared.drop(columns='views')
df_prepared = df_prepared.drop(columns='comments')
df_prepared.head()

,channel,text,date,media,reaction_😢,reaction_🔥,reaction_🤨,reaction_🤡,reaction_🕊,reaction_🎉,...,tag_Армия,tag_Путин,tag_Спорт,tag_Интернет,tag_НАТО,tag_Цены,tag_ООН,tag_Искусственный интеллект,normalized_views,normalized_comments
0,@novosti_voinaa,У Илона Маска появился 14-й ребенок. Еще одног...,2025-03-01 11:40:08,PHOTO,0.000000,0.508475,0.677966,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.000000,0.034757
1,@novosti_voinaa,"Восстановить бонусные карты IKEA, ZARA, LUSH и...",2025-03-01 10:18:01,PHOTO,2.178649,0.217865,13.289760,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.079088,0.014482
2,@novosti_voinaa,И для полноты картины минувшего и надолго запо...,2025-03-01 00:09:40,PHOTO,4.669647,5.414804,1.788376,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.268185,0.118030
3,@novosti_voinaa,"Видео, после которого вы начнете крепче завязы...",2025-02-28 23:59:01,VIDEO,7.564576,0.738007,0.461255,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.227074,0.018103
5,@novosti_voinaa,Таких адреналиновых официальных переговоров гл...,2025-02-28 23:08:49,PHOTO,0.000000,1.249082,2.718589,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0.225074,0.106445
